In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, InputLayer, Input, add, dot, maximum, average, multiply, concatenate, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
import numpy as np
import pandas as pd

In [2]:
# reading data
data = pd.read_csv('PrOCTOR_sample_data_all.csv', header=0)
data1 = data.fillna(data.mean()['MolecularWeight':'Salivary Gland'])
data1["target"] = np.where(data1.iloc[:, 1] == "passed", 1, 0)

In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# x, y variable
X = data1.iloc[:, 2:-1]
y = data1['target']
#print(X)

In [5]:
X = np.array(X)
X = scaler.fit_transform(X)
y = np.array(y)
y_1 = y.reshape(y.shape[0],-1)
y_train = y_1
#y = scaler.fit_transform(X)
des=X[:,:13]
body=X[:,13:]

In [6]:
hid_layer = [100, 100, 50]
print(hid_layer[-1])

50


In [7]:
_, des_col = des.shape
_, body_col = body.shape

In [8]:
class early_Concat:
    def __init__(self, des_shape, body_shape, hid_layer, output_layer):
        self.des_shape = des_shape
        self.body_shape = body_shape
        self.hid_layer = hid_layer  # not used
        self.output_layer = output_layer
        
    def Multi(self):
        # des modality
        des_input = Input(shape = (self.des_shape)) # input = 17, des_shape = 17
        body_input = Input(shape = (self.body_shape)) # input = 30,body_shape = 30
        bias_input = Input(shape = (1,))
        
        des_reshape = Dense(50)(des_input)
        body_reshape = Dense(50)(body_input)
        
        concat_1 = concatenate([des_reshape , bias_input])
        concat_2 = concatenate([body_reshape , bias_input])
        
        des_reshape_2 = Reshape((1,51))(concat_1)
        body_reshape_2 = Reshape((1,51))(concat_2)
        
        dot_layer = dot([des_reshape_2, body_reshape_2], axes = 1, normalize=False)
        tensor_flat = Flatten()(dot_layer)
        #x = Dense(self.des_shape + self.body_shape , activation = 'relu')(concat)
        #x = BatchNormalization()(x)
        
        x = Dense(hid_layer[0] , activation = 'relu')(tensor_flat)
        x = BatchNormalization()(x)

        x = Dense(hid_layer[1] , activation = 'relu')(x)
        x = BatchNormalization()(x)

        x = Dense(hid_layer[2] , activation = 'relu')(x)
        x = BatchNormalization()(x)
        
        x = Dense(self.output_layer , activation = 'sigmoid')(x)
        
        early_fusion = keras.models.Model(inputs = [bias_input, des_input, body_input], outputs = [x])

        return early_fusion

In [9]:
model = early_Concat(des_col, body_col, hid_layer, 1)
earlyfusion= model.Multi()
print(earlyfusion)

In [10]:
earlyfusion.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 13)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 34)]         0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 50)           700         input_1[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 1)]          0                                            
______________________________________________________________________________________________

In [11]:
from sklearn.model_selection import LeaveOneOut, KFold, cross_val_score
from imblearn.over_sampling import SMOTE
from sklearn.utils.class_weight import compute_class_weight

# 모델설정
sm = SMOTE(random_state=202004)
loo = LeaveOneOut()
kfold = KFold(n_splits = 10, shuffle = True, random_state = 111)
kfold.get_n_splits(X)

10

In [12]:
def predict_change(predict):
    y_predict = []
    for i in range(len(predict)):
        y_predict = np.append(y_predict, predict[i])
    
    y_predict = np.array(y_predict)
    y_predict = y_predict.reshape(y_predict.shape[0],-1)
    return y_predict

In [13]:
dict_y = dict()

class train_model:
    def __init__(self, x, y, model, pd_y, pd_sm_y, pd_cw_y, pd_smcw_y):
        self.X = x
        self.y = y
        self.model = model
        self.pd_y = pd_y
        self.pd_sm_y = pd_sm_y
        self.pd_cw_y = pd_cw_y
        self.pd_smcw_y = pd_smcw_y
        
    def train_based(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            des_train, des_test = des[train_index], des[test_index]
            body_train, body_test = body[train_index], body[test_index]
            y_train, y_test = y[train_index], y[test_index]

            self.model.compile(optimizer= keras.optimizers.Adam(), loss='binary_crossentropy', metrics = ['accuracy'])
            print(des_train.shape[0])
            kf_history = self.model.fit(x = [np.ones(des_train.shape[0]),des_train, body_train], y = y_train, epochs=50)

            y_pred = self.model.predict([np.ones(des_test.shape[0]), des_test[:], body_test[:]])

            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_y[index] = y_pred[i]
            
        return self.pd_y
    
    def train_smote(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            des_train, des_test = des[train_index], des[test_index]
            body_train, body_test = body[train_index], body[test_index]
            y_train, y_test = y[train_index], y[test_index]
            
            sm_des_train, sm_y_train = sm.fit_sample(des_train, y_train)
            sm_body_train, _ = sm.fit_sample(body_train, y_train)

            self.model.compile(optimizer= keras.optimizers.Adam(), loss='binary_crossentropy', metrics = ['accuracy'])
            kf_history = self.model.fit(x = [sm_des_train, sm_body_train], y = sm_y_train, epochs=50)

            y_pred = self.model.predict([des_test[:], body_test[:]])

            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_y[index] = y_pred[i]
            
        return self.pd_sm_y
    
    def train_weight(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            des_train, des_test = des[train_index], des[test_index]
            body_train, body_test = body[train_index], body[test_index]
            y_train, y_test = y[train_index], y[test_index]

            neg, pos = np.bincount(y_train)
            total = neg + pos

            weight_for_0 = (1 / neg)*(total)/2.0 
            weight_for_1 = (1 / pos)*(total)/2.0

            class_weight = {0: weight_for_0, 1: weight_for_1}
            
            self.model.compile(optimizer= Adam(), loss='binary_crossentropy', metrics = ['accuracy'])
            kf_history = self.model.fit(x = [np.ones(des_train.shape[0]), des_train, body_train], y = y_train, epochs=50, class_weight = class_weight)

            y_pred = self.model.predict([np.ones(des_test.shape[0]), des_test[:], body_test[:]])

            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_y[index] = y_pred[i]
            
        return self.pd_cw_y
    
    def train_sm_weight(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            des_train, des_test = des[train_index], des[test_index]
            body_train, body_test = body[train_index], body[test_index]
            y_train, y_test = y[train_index], y[test_index]

            neg, pos = np.bincount(y_train)
            total = neg + pos

            weight_for_0 = (1 / neg)*(total)/2.0 
            weight_for_1 = (1 / pos)*(total)/2.0

            class_weight = {0: weight_for_0, 1: weight_for_1}
    
            sm_des_train, sm_y_train = sm.fit_sample(des_train, y_train)
            sm_body_train, _ = sm.fit_sample(body_train, y_train)
    
            self.model.compile(optimizer= Adam(), loss='binary_crossentropy', metrics = ['accuracy'])
            kf_history = self.model.fit(x = [np.ones(sm_des_train.shape[0]), sm_des_train, sm_body_train], y = sm_y_train, epochs=50, class_weight = class_weight)

            y_pred = self.model.predict([np.ones(des_test.shape[0]), des_test[:], body_test[:]])

            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_y[index] = y_pred[i]
            
        return self.pd_smcw_y

In [1]:
model = early_Concat(des_col, body_col, hid_layer, 1)
earlyfusion= model.Multi()
#print(earlyfusion)

early_y, early_sm_y, early_cw_y, early_smcw_y = [], [], [], []
output_early = train_model(X, y, earlyfusion, dict_y, dict_y, dict_y, dict_y)

# sum model training
early_y = output_early.train_based()
#early_sm_y = output_early.train_smote()
#early_cw_y = output_early.train_weight()
#early_smcw_y = output_early.train_sm_weight()

In [51]:
from sklearn.metrics import roc_curve, precision_recall_curve, auc, f1_score, confusion_matrix, accuracy_score, matthews_corrcoef, accuracy_score
from imblearn.metrics import geometric_mean_score
import math

class Evaluation:
    def __init__(self, pred, y):
        self.y_pred = pred
        self.y = y
        
    def matrix(self):
        y_1 = self.y.reshape(self.y.shape[0],-1)
        y_pred = np.array(self.y_pred)
        y_pred = y_pred.reshape(y_pred.shape[0], -1)
        
        y_classify = []
        for i in range(len(self.y_pred)):
            if self.y_pred[i] >= 0.5:
                a = 1.
                y_classify.append(a)
            else:
                a = 0.
                y_classify.append(a)
        
        fpr,tpr,threshold = roc_curve(y_1 , y_pred, pos_label = 1)
        precision, recall, threshold = precision_recall_curve(y_1, y_pred, pos_label = 1)
        
        roc_auc = auc(fpr,tpr)
        auprc = auc(recall, precision)
        mean_precision = np.mean(precision)
        mean_recall = np.mean(recall)
        F1 = 2 * (mean_precision * mean_recall) / (mean_precision + mean_recall)
        # binary 
        accuracy = accuracy_score(y_1, y_classify)
        mcc = matthews_corrcoef(y_1, y_classify) 
        g_mean = geometric_mean_score(y_1, y_classify)
        confusion = confusion_matrix(y_1, y_classify)
        
        tn, fp, fn, tp = confusion.ravel()
        tpr = tp / (tp + fn)
        tnr = tn / (tn + fp)
        ppv = tp / (tp + fp)
        fnr = fn / (fn + tp)
        fpr = fp / (fp + tn)

        confu_precision = ppv 
        confu_recall = tpr # sensitivity
        confu_f1 = 2 * ((ppv * tpr) / (ppv + tpr))
        confu_accuracy = (tp + tn) / (tp + tn + fp + fn)
        confu_mcc = ((tp * tn)-(fp-fn))/ math.sqrt((tp + fp)*(tp + fn)*(tn + fp)*(tn + fn))
        confu_g_mean = math.sqrt(tpr * tnr)
        Optimized_precision = (confu_accuracy - abs(tnr-tpr)) / (tnr + tpr)

        print('공통 \nAUC :',roc_auc) # pb
        print("AUPRC :", auprc) # pb
        print("Optimized precision :", Optimized_precision)

        print("\nfunction 사용\nAccuracy :", accuracy)
        print("Precision(pb) :", mean_precision) #pb
        print("Recall(pb) :", mean_recall) # pb
        print("F1 score(pb) :", F1) #pb
        print("MCC :", mcc)
        print("G-mean :", g_mean)
        
        print("\nConfusion_matrix 사용 \n", confusion)
        print("Accuracy :", confu_accuracy)
        print("Precision :", confu_precision) 
        print("Recall :", confu_recall) 
        print("F1 score :", confu_f1) 
        print("MCC :", confu_mcc)
        print("G-mean :", confu_g_mean)
        
        return roc_auc, auprc, Optimized_precision, accuracy, mean_precision, mean_recall, F1, mcc, g_mean, confu_accuracy, confu_precision, confu_recall, confu_f1, confu_mcc, confu_g_mean;

In [61]:
early_y_result = Evaluation(predict_change(early_y), y_1)
#early_sm_y_result = Evaluation(early_sm_y, y)
#early_cw_y_result = Evaluation(predict_change(early_cw_y), y)
#early_smcw_y_result = Evaluation(early_smcw_y, y)

print("#early model 결과 \n")
print("\n-----based model-----\n")
print(early_y_result.matrix())

print("\n-----smote model-----\n")
#print(early_sm_y_result.matrix())

print("\n-----class weight model-----\n")
#print(early_cw_y_result.matrix())

print("\n-----smote + class weight model-----\n")
#print(early_smcw_y_result.matrix())

#early model 결과 


-----based model-----

공통 
AUC : 0.9736543434982418
AUPRC : 0.9972554984561129
Optimized precision : 0.4282977054094722

function 사용
Accuracy : 0.9722222222222222
Precision(pb) : 0.9951423574463145
Recall(pb) : 0.6567587388114787
F1 score(pb) : 0.7912924582408878
MCC : 0.814246368234619
G-mean : 0.87606203936724

Confusion_matrix 사용 
 [[ 55  16]
 [  7 750]]
Accuracy : 0.9722222222222222
Precision : 0.97911227154047
Recall : 0.9907529722589168
F1 score : 0.9848982271831911
MCC : 0.8162850520774935
G-mean : 0.87606203936724
(0.9736543434982418, 0.9972554984561129, 0.4282977054094722, 0.9722222222222222, 0.9951423574463145, 0.6567587388114787, 0.7912924582408878, 0.814246368234619, 0.87606203936724, 0.9722222222222222, 0.97911227154047, 0.9907529722589168, 0.9848982271831911, 0.8162850520774935, 0.87606203936724)

-----smote model-----


-----class weight model-----


-----smote + class weight model-----



In [72]:
early_y_set = []

for i in range(20):
    model = early_Concat(des_col, body_col, hid_layer, 1)
    earlyfusion= model.Multi()
    print(earlyfusion)

    early_y, early_sm_y, early_cw_y, early_smcw_y = [], [], [], []
    output_early = train_model(X, y, earlyfusion, dict_y, dict_y, dict_y, dict_y)

    # concatenate model training
    #early_y = output_early.train_based()
    #early_sm_y = output_early.train_smote()
    early_cw_y = output_early.train_weight()
    #early_smcw_y = output_early.train_sm_weight()
    
    #early_y_result = Evaluation(predict_change(early_y), y_1)
    #early_sm_y_result = Evaluation(early_sm_y, y)
    early_cw_y_result = Evaluation(predict_change(early_cw_y), y)
    #early_smcw_y_result = Evaluation(predict_change(early_smcw_y), y)
    
    early_y_set = np.append(early_y_set, early_cw_y_result.matrix())
    print(len(early_y_set))
    
early_y_np = np.array(early_y_set)

early_y_np = early_y_np.reshape(-1,15)
early_y_np.shape

TEST: [  1  11  12  34  36  49  56  71  93 112 119 125 126 140 145 152 157 183
 190 194 198 205 209 210 211 214 233 237 257 264 272 298 303 323 328 340
 354 368 373 380 399 403 409 412 422 432 434 447 456 461 464 468 474 476
 491 496 506 518 520 523 528 532 538 547 553 556 589 611 624 629 645 658
 665 700 702 708 735 757 766 774 799 800 825]
Epoch 1/50
24/24 [==============================] - 0s 2ms/step - loss: 0.8496 - accuracy: 0.4644
Epoch 2/50
24/24 [==============================] - 0s 2ms/step - loss: 0.5635 - accuracy: 0.5557
Epoch 3/50
24/24 [==============================] - 0s 2ms/step - loss: 0.4725 - accuracy: 0.6430
Epoch 4/50
24/24 [==============================] - 0s 2ms/step - loss: 0.4106 - accuracy: 0.7034
Epoch 5/50
24/24 [==============================] - 0s 3ms/step - loss: 0.3798 - accuracy: 0.7772
Epoch 6/50
24/24 [==============================] - 0s 3ms/step - loss: 0.3823 - accuracy: 0.7919
Epoch 7/50
24/24 [==============================] - 0s 2ms/step - lo

24/24 [==============================] - 0s 2ms/step - loss: 0.0424 - accuracy: 0.9772
Epoch 3/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0430 - accuracy: 0.9852
Epoch 4/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0341 - accuracy: 0.9852
Epoch 5/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0353 - accuracy: 0.9866
Epoch 6/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0320 - accuracy: 0.9852
Epoch 7/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0318 - accuracy: 0.9879
Epoch 8/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0254 - accuracy: 0.9866
Epoch 9/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0200 - accuracy: 0.9893
Epoch 10/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0148 - accuracy: 0.9960
Epoch 11/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0160 - accuracy: 0.9960
Epoch 12/50
24/24 [==========

24/24 [==============================] - 0s 2ms/step - loss: 0.0154 - accuracy: 0.9933
Epoch 8/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0152 - accuracy: 0.9933
Epoch 9/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0130 - accuracy: 0.9933
Epoch 10/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0098 - accuracy: 0.9960
Epoch 11/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0103 - accuracy: 0.9960
Epoch 12/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0076 - accuracy: 0.9960
Epoch 13/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0052 - accuracy: 0.9973
Epoch 14/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0082 - accuracy: 0.9973
Epoch 15/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0136 - accuracy: 0.9946
Epoch 16/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0171 - accuracy: 0.9933
Epoch 17/50
24/24 [=====

Epoch 12/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0299 - accuracy: 0.9960
Epoch 13/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0194 - accuracy: 0.9933
Epoch 14/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0140 - accuracy: 0.9973
Epoch 15/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0105 - accuracy: 0.9960
Epoch 16/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0079 - accuracy: 0.9973
Epoch 17/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0038 - accuracy: 1.0000
Epoch 18/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0068 - accuracy: 0.9973
Epoch 19/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0067 - accuracy: 0.9973
Epoch 20/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0021 - accuracy: 1.0000
Epoch 21/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0031 - accuracy: 0.9987
Epoch 22/5

24/24 [==============================] - 0s 2ms/step - loss: 0.0320 - accuracy: 0.9839
Epoch 12/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0210 - accuracy: 0.9919
Epoch 13/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0193 - accuracy: 0.9946
Epoch 14/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0180 - accuracy: 0.9919
Epoch 15/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0168 - accuracy: 0.9933
Epoch 16/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0335 - accuracy: 0.9919
Epoch 17/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0226 - accuracy: 0.9919
Epoch 18/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0165 - accuracy: 0.9933
Epoch 19/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0183 - accuracy: 0.9893
Epoch 20/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0480 - accuracy: 0.9852
Epoch 21/50
24/24 [===

...

24/24 [==============================] - 0s 2ms/step - loss: 0.0340 - accuracy: 0.9879
Epoch 21/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0523 - accuracy: 0.9893
Epoch 22/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0405 - accuracy: 0.9852
Epoch 23/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0772 - accuracy: 0.9745
Epoch 24/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0724 - accuracy: 0.9732
Epoch 25/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0524 - accuracy: 0.9732
Epoch 26/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0341 - accuracy: 0.9866
Epoch 27/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0357 - accuracy: 0.9906
Epoch 28/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0316 - accuracy: 0.9906
Epoch 29/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0253 - accuracy: 0.9933
Epoch 30/50
24/24 [===

24/24 [==============================] - 0s 2ms/step - loss: 0.0186 - accuracy: 0.9946
Epoch 26/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0167 - accuracy: 0.9919
Epoch 27/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0203 - accuracy: 0.9933
Epoch 28/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0143 - accuracy: 0.9946
Epoch 29/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0267 - accuracy: 0.9960
Epoch 30/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0225 - accuracy: 0.9906
Epoch 31/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0214 - accuracy: 0.9919
Epoch 32/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0379 - accuracy: 0.9919
Epoch 33/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0435 - accuracy: 0.9812
Epoch 34/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0202 - accuracy: 0.9893
Epoch 35/50
24/24 [===

24/24 [==============================] - 0s 2ms/step - loss: 0.0341 - accuracy: 0.9879
Epoch 2/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0241 - accuracy: 0.9919
Epoch 3/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0156 - accuracy: 0.9839
Epoch 4/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0185 - accuracy: 0.9933
Epoch 5/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0194 - accuracy: 0.9919
Epoch 6/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0097 - accuracy: 0.9973
Epoch 7/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0071 - accuracy: 0.9987
Epoch 8/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0072 - accuracy: 0.9973
Epoch 9/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0077 - accuracy: 0.9987
Epoch 10/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0085 - accuracy: 0.9946
Epoch 11/50
24/24 [===========

24/24 [==============================] - 0s 2ms/step - loss: 0.0079 - accuracy: 0.9987
Epoch 7/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0056 - accuracy: 0.9973
Epoch 8/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0044 - accuracy: 0.9987
Epoch 9/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0036 - accuracy: 0.9987
Epoch 10/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0065 - accuracy: 0.9973
Epoch 11/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0038 - accuracy: 0.9987
Epoch 12/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0119 - accuracy: 0.9960
Epoch 13/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0099 - accuracy: 0.9973
Epoch 14/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0033 - accuracy: 0.9987
Epoch 15/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0040 - accuracy: 0.9973
Epoch 16/50
24/24 [======

(20, 15)

In [58]:
def result(data):
    auc = data[:,0]
    auprc = data[:,1]
    op = data[:,2]
    acc = data[:,3]
    precision = data[:,4]
    recall = data[:,5]
    f1 = data[:,6]
    mcc = data[:,7]
    g_mean = data[:,8]
    confu_acc = data[:,9]
    confu_precision = data[:,10]
    confu_recall = data[:,11]
    confu_f1 = data[:,12]
    confu_mcc = data[:,13]
    confu_g_mean = data[:,14]

    print('공통 \nAUC :\n',"[평균 :",format(auc.mean(), ".5f"), "] [표준편차 :",format(auc.std(), ".5f"),"]") # pb
    print("AUPRC :\n", "[평균 :",format(auprc.mean(), ".5f"), "] [표준편차 :",format(auprc.std(), ".5f"),"]")
    print("Optimized precision :\n","[평균 :",format(op.mean(), ".5f"), "] [표준편차 :",format(op.std(), ".5f"),"]")

    print("\nfunction 사용\nAccuracy :\n","[평균 :",format(acc.mean(), ".5f"), "] [표준편차 :",format(acc.std(), ".5f"),"]")
    print("Precision(pb) :\n", "[평균 :",format(precision.mean(), ".5f"), "] [표준편차 :",format(precision.std(), ".5f"),"]")
    print("Recall(pb) :\n", "[평균 :",format(recall.mean(), ".5f"), "] [표준편차 :",format(recall.std(), ".5f"),"]")
    print("F1 score(pb) :\n","[평균 :",format(f1.mean(), ".5f"), "] [표준편차 :",format(f1.std(), ".5f"),"]")
    print("MCC :\n", "[평균 :",format(mcc.mean(), ".5f"), "] [표준편차 :",format(mcc.std(), ".5f"),"]")
    print("G-mean :\n", "[평균 :",format(g_mean.mean(), ".5f"), "] [표준편차 :",format(g_mean.std(), ".5f"),"]")
    
    print("\nConfusion matrix 사용 \nAccuracy :\n", "[평균 :",format(confu_acc.mean(), ".5f"), "] [표준편차 :",format(confu_acc.std(), ".5f"),"]")
    print("Precision :\n", "[평균 :",format(confu_precision.mean(), ".5f"), "] [표준편차 :",format(confu_precision.std(), ".5f"),"]")
    print("Recall :\n", "[평균 :",format(confu_recall.mean(), ".5f"), "] [표준편차 :",format(confu_recall.std(), ".5f"),"]")
    print("F1 score :\n", "[평균 :",format(confu_f1.mean(), ".5f"), "] [표준편차 :",format(confu_f1.std(), ".5f"),"]")
    print("MCC :\n", "[평균 :",format(confu_mcc.mean(), ".5f"), "] [표준편차 :",format(confu_mcc.std(), ".5f"),"]")
    print("G-mean :\n", "[평균 :",format(confu_g_mean.mean(), ".5f"), "] [표준편차 :",format(confu_g_mean.std(), ".5f"),"]")
    

In [73]:
result(early_y_np)

공통 
AUC :
 [평균 : 0.96627 ] [표준편차 : 0.00718 ]
AUPRC :
 [평균 : 0.99627 ] [표준편차 : 0.00092 ]
Optimized precision :
 [평균 : 0.45042 ] [표준편차 : 0.01086 ]

function 사용
Accuracy :
 [평균 : 0.95121 ] [표준편차 : 0.00723 ]
Precision(pb) :
 [평균 : 0.99314 ] [표준편차 : 0.00137 ]
Recall(pb) :
 [평균 : 0.58375 ] [표준편차 : 0.01236 ]
F1 score(pb) :
 [평균 : 0.73522 ] [표준편차 : 0.00957 ]
MCC :
 [평균 : 0.71880 ] [표준편차 : 0.03398 ]
G-mean :
 [평균 : 0.88576 ] [표준편차 : 0.01725 ]

Confusion matrix 사용 
Accuracy :
 [평균 : 0.95121 ] [표준편차 : 0.00723 ]
Precision :
 [평균 : 0.98224 ] [표준편차 : 0.00283 ]
Recall :
 [평균 : 0.96407 ] [표준편차 : 0.00702 ]
F1 score :
 [평균 : 0.97306 ] [표준편차 : 0.00408 ]
MCC :
 [평균 : 0.72521 ] [표준편차 : 0.03237 ]
G-mean :
 [평균 : 0.88576 ] [표준편차 : 0.01725 ]
